In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models

In [2]:

ruta_data = 'DatasetFuegoHumo/data'

In [ ]:

ruta_train_imagenes = os.path.join(ruta_data, "train", "images")
ruta_val_imagenes = os.path.join(ruta_data, "val", "images")


def contar_escala_grises(ruta_imagenes, nombre_carpeta, max_imagenes=None):
    total_imagenes = 0
    grises = 0
    
    lista_imagenes = [f for f in os.listdir(ruta_imagenes) if f.endswith(('.jpg', '.png', '.jpeg'))]
    if max_imagenes:
        lista_imagenes = lista_imagenes[:max_imagenes]
    
    for nombre_imagen in lista_imagenes:
        ruta_imagen = os.path.join(ruta_imagenes, nombre_imagen)
        imagen = Image.open(ruta_imagen)
        
    
        if imagen.mode != 'RGB':
            grises += 1
            print(f"{nombre_carpeta} - Imagen en escala de grises (o no RGB): {nombre_imagen} - Modo: {imagen.mode}")
        
        total_imagenes += 1
    
    print(f"\n{nombre_carpeta}:")
    print(f"Total de imágenes analizadas: {total_imagenes}")
    print(f"Imágenes en escala de grises (o no RGB): {grises}")
    print(f"Porcentaje en escala de grises: {(grises / total_imagenes * 100):.2f}%")

contar_escala_grises(ruta_train_imagenes, "Train", max_imagenes=300) 
contar_escala_grises(ruta_val_imagenes, "Val")  


Train:
Total de imágenes analizadas: 300
Imágenes en escala de grises (o no RGB): 0
Porcentaje en escala de grises: 0.00%

Val:
Total de imágenes analizadas: 3096
Imágenes en escala de grises (o no RGB): 0
Porcentaje en escala de grises: 0.00%


In [4]:




def cargar_datos(ruta_imagenes, ruta_labels, max_imagenes=None):
    imagenes = []
    etiquetas = []
    
    lista_imagenes = [f for f in os.listdir(ruta_imagenes) if f.endswith(('.jpg', '.png', '.jpeg'))]
    if max_imagenes:
        lista_imagenes = lista_imagenes[:max_imagenes]
    
    for nombre_imagen in lista_imagenes:
        
        ruta_imagen = os.path.join(ruta_imagenes, nombre_imagen)
        imagen = Image.open(ruta_imagen).resize((128, 128))
        imagen = np.array(imagen) / 255.0  # Normaliza a [0, 1]
        
        # Etiqueta basada en .txt (solo para entrenamiento)
        nombre_base = os.path.splitext(nombre_imagen)[0]
        ruta_txt = os.path.join(ruta_labels, f"{nombre_base}.txt")
        if os.path.exists(ruta_txt):
            with open(ruta_txt, 'r', encoding='utf-8') as archivo:
                contenido = archivo.read().strip()
                etiqueta = 1 if contenido else 0  # 1 si hay fuego/humo, 0 si no
        else:
            etiqueta = 0
        
        imagenes.append(imagen)
        etiquetas.append(etiqueta)
    
    return np.array(imagenes), np.array(etiquetas)


In [5]:
# Carga datos de entrenamiento y validación
ruta_train_imagenes = os.path.join(ruta_data, "train", "images")
ruta_train_labels = os.path.join(ruta_data, "train", "labels")
ruta_val_imagenes = os.path.join(ruta_data, "val", "images")
ruta_val_labels = os.path.join(ruta_data, "val", "labels")


In [6]:
X_train, y_train = cargar_datos(ruta_train_imagenes, ruta_train_labels, max_imagenes=900)
X_val, y_val = cargar_datos(ruta_val_imagenes, ruta_val_labels)

In [7]:
# Define el modelo CNN 
modelo = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),  # Más filtros
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),  # Capa adicional
    layers.Flatten(),
    layers.Dense(128, activation='relu'),           
    layers.Dropout(0.5),                            # Evita sobreajuste
    layers.Dense(1, activation='sigmoid')           # Salida binaria
])

c:\Users\Quimey\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:

modelo.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [10]:

modelo.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), verbose=1)

Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 142s 5s/step - accuracy: 0.9062 - loss: 0.2069 - val_accuracy: 0.4438 - val_loss: 10.7331
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.9961 - loss: 0.0424 - val_accuracy: 0.4438 - val_loss: 4.9285
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.9948 - loss: 0.0514 - val_accuracy: 0.4438 - val_loss: 5.5852
Epoch 4/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.9953 - loss: 0.0374 - val_accuracy: 0.4438 - val_loss: 3.6862
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.9972 - loss: 0.0260 - val_accuracy: 0.4438 - val_loss: 4.3115
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.9944 - loss: 0.0419 - val_accuracy: 0.4438 - val_loss: 4.7563
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 43s 1s/step - accuracy: 0.9968 - loss: 0.0264 - val_accuracy: 0.4438 - val_loss: 3.3855
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9937 - loss: 0.0413 - val_accuracy: 0.4438 - val_los

In [11]:
accuracy = modelo.evaluate(X_val, y_val, verbose=0)[1]
accuracy_porcentaje = accuracy * 100

print("\nResultados en validación:")
print(f"Precisión (Accuracy): {accuracy_porcentaje:.2f}%")


Resultados en validación:
Precisión (Accuracy): 44.38%


In [8]:
# Función para contar etiquetas
def contar_etiquetas(ruta_imagenes, ruta_labels, nombre_carpeta, max_imagenes=None):
    positivos = 0  # Fuego/humo
    negativos = 0  # Sin fuego/humo
    
    # Lista de imágenes
    lista_imagenes = [f for f in os.listdir(ruta_imagenes) if f.endswith(('.jpg', '.png', '.jpeg'))]
    if max_imagenes:
        lista_imagenes = lista_imagenes[:max_imagenes]
    
    total_imagenes = len(lista_imagenes)
    
    for nombre_imagen in lista_imagenes:
        nombre_base = os.path.splitext(nombre_imagen)[0]
        ruta_txt = os.path.join(ruta_labels, f"{nombre_base}.txt")
        
        if os.path.exists(ruta_txt):
            with open(ruta_txt, 'r', encoding='utf-8') as archivo:
                contenido = archivo.read().strip()
                if contenido:  # Si hay coordenadas
                    positivos += 1
                else:  # Si está vacío
                    negativos += 1
        else:  # Si no hay .txt
            negativos += 1
    print(f"\n{nombre_carpeta}:")
    print(f"Imágenes con fuego/humo (1): {positivos}")
    print(f"Imágenes sin fuego/humo (0): {negativos}")
    print(f"Total de imágenes: {total_imagenes}")
    print(f"Porcentaje de fuego/humo: {(positivos / total_imagenes * 100):.2f}%")
    print(f"Porcentaje sin fuego/humo: {(negativos / total_imagenes * 100):.2f}%")

    # Conta en train y val
contar_etiquetas(ruta_train_imagenes, ruta_train_labels, "Train", max_imagenes=300)
contar_etiquetas(ruta_val_imagenes, ruta_val_labels, "Val")







Train:
Imágenes con fuego/humo (1): 2
Imágenes sin fuego/humo (0): 298
Total de imágenes: 300
Porcentaje de fuego/humo: 0.67%
Porcentaje sin fuego/humo: 99.33%

Val:
Imágenes con fuego/humo (1): 1722
Imágenes sin fuego/humo (0): 1374
Total de imágenes: 3096
Porcentaje de fuego/humo: 55.62%
Porcentaje sin fuego/humo: 44.38%


In [9]:
ruta_train_imagenes = os.path.join(ruta_data, "train", "images")
ruta_train_labels = os.path.join(ruta_data, "train", "labels")
ruta_val_imagenes = os.path.join(ruta_data, "val", "images")
ruta_val_labels = os.path.join(ruta_data, "val", "labels")

# Sin límite en train para usar todas las imágenes
X_train, y_train = cargar_datos(ruta_train_imagenes, ruta_train_labels, max_imagenes=1200)
X_val, y_val = cargar_datos(ruta_val_imagenes, ruta_val_labels)




In [10]:
# Calcula pesos de clase para balancear
num_positivos = np.sum(y_train == 1)
num_negativos = np.sum(y_train == 0)
peso_positivo = num_negativos / num_positivos if num_positivos > 0 else 1
class_weights = {0: 1.0, 1: peso_positivo}

print(f"Train - Positivos: {num_positivos}, Negativos: {num_negativos}")
print(f"Peso para clase positiva: {peso_positivo:.2f}")

Train - Positivos: 5, Negativos: 1195
Peso para clase positiva: 239.00


In [11]:
from tensorflow.keras.applications import MobileNetV2

In [12]:

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

modelo = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [17]:

modelo.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [18]:
# Entrena el modelo con pesos de clase
modelo.fit(X_train, y_train, epochs=3, batch_size=32, 
           validation_data=(X_val, y_val), class_weight=class_weights, verbose=1)

Epoch 1/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 192s 4s/step - accuracy: 0.9210 - loss: 8.0286 - val_accuracy: 0.4438 - val_loss: 2.8329
Epoch 2/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.6872 - loss: 3.1118 - val_accuracy: 0.4648 - val_loss: 1.6810
Epoch 3/3
38/38 ━━━━━━━━━━━━━━━━━━━━ 58s 2s/step - accuracy: 0.6223 - loss: 1.2221 - val_accuracy: 0.4302 - val_loss: 2.9581


In [19]:
# Calcula la accuracy
accuracy = modelo.evaluate(X_val, y_val, verbose=0)[1]
accuracy_porcentaje = accuracy * 100

print("\nResultados en validación:")
print(f"Precisión (Accuracy): {accuracy_porcentaje:.2f}%")


Resultados en validación:
Precisión (Accuracy): 43.02%


In [12]:
# Separa imágenes positivas y negativas
positivas_idx = np.where(y_train == 1)[0]
negativas_idx = np.where(y_train == 0)[0]
X_positivas = X_train[positivas_idx]
X_negativas = X_train[negativas_idx]

print(f"Train - Positivos originales: {len(X_positivas)}, Negativos: {len(X_negativas)}")



Train - Positivos originales: 5, Negativos: 1195


In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [14]:
# Aumentación para imágenes positivas
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [15]:
# Genera más imágenes positivas hasta igualar (o acercarse) a las negativas
X_positivas_aumentadas = []
y_positivas_aumentadas = []
target_positivas = min(len(X_negativas), len(X_positivas) * 50)  # Límite razonable
for i in range(len(X_positivas)):
    img = X_positivas[i]
    img = img.reshape((1,) + img.shape)  # Añade dimensión batch
    for batch in datagen.flow(img, batch_size=1):
        X_positivas_aumentadas.append(batch[0])
        y_positivas_aumentadas.append(1)
        if len(X_positivas_aumentadas) >= target_positivas:
            break

# Combina datos originales y aumentados
X_train_balanced = np.concatenate([X_negativas, X_positivas, np.array(X_positivas_aumentadas)])
y_train_balanced = np.concatenate([np.zeros(len(X_negativas)), np.ones(len(X_positivas)), np.ones(len(X_positivas_aumentadas))])

print(f"Train - Positivos después de aumentación: {len(y_train_balanced[y_train_balanced == 1])}, Negativos: {len(y_train_balanced[y_train_balanced == 0])}")

Train - Positivos después de aumentación: 259, Negativos: 1195


In [17]:
# Modelo con MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

modelo = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [25]:
# Compila el modelo
modelo.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [26]:
# Entrena el modelo
modelo.fit(X_train_balanced, y_train_balanced, epochs=2, batch_size=32, validation_data=(X_val, y_val), verbose=1)


Epoch 1/2
46/46 ━━━━━━━━━━━━━━━━━━━━ 153s 3s/step - accuracy: 0.8504 - loss: 0.4105 - val_accuracy: 0.4612 - val_loss: 2.6000
Epoch 2/2
46/46 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.9963 - loss: 0.0270 - val_accuracy: 0.4968 - val_loss: 1.7221


In [27]:
# Calcula la accuracy
accuracy = modelo.evaluate(X_val, y_val, verbose=0)[1]
accuracy_porcentaje = accuracy * 100

print("\nResultados en validación:")
print(f"Precisión (Accuracy): {accuracy_porcentaje:.2f}%")


Resultados en validación:
Precisión (Accuracy): 49.68%


In [16]:
# Separa imágenes positivas y negativas
positivas_idx = np.where(y_train == 1)[0]
negativas_idx = np.where(y_train == 0)[0]
X_positivas = X_train[positivas_idx]
X_negativas = X_train[negativas_idx]

print(f"Train - Positivos originales: {len(X_positivas)}, Negativos originales: {len(X_negativas)}")

Train - Positivos originales: 5, Negativos originales: 1195


In [17]:
# Toma solo 400 negativos 
np.random.seed(42)  
negativas_seleccionadas_idx = np.random.choice(negativas_idx, size=400, replace=False)
X_negativas_seleccionadas = X_train[negativas_seleccionadas_idx]
# Aumentación para llegar a 300 positivos
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

X_positivas_aumentadas = []
y_positivas_aumentadas = []
target_positivas = 300  
for i in range(len(X_positivas)):
    img = X_positivas[i]
    img = img.reshape((1,) + img.shape)
    count = 0
    for batch in datagen.flow(img, batch_size=1):
        X_positivas_aumentadas.append(batch[0])
        y_positivas_aumentadas.append(1)
        count += 1
        if len(X_positivas_aumentadas) >= target_positivas - len(X_positivas):  # Restamos los originales
            break
    while len(X_positivas_aumentadas) < target_positivas - len(X_positivas) and count < 100:
        for batch in datagen.flow(img, batch_size=1):
            X_positivas_aumentadas.append(batch[0])
            y_positivas_aumentadas.append(1)
            count += 1
            if len(X_positivas_aumentadas) >= target_positivas - len(X_positivas):
                break

In [18]:
# Combina datos: 400 negativos + 4 originales + 299 aumentados = 700 total
X_train_balanced = np.concatenate([X_negativas_seleccionadas, X_positivas, np.array(X_positivas_aumentadas)])
y_train_balanced = np.concatenate([np.zeros(400), np.ones(len(X_positivas)), np.ones(len(X_positivas_aumentadas))])

print(f"Train - Positivos después de aumentación: {len(y_train_balanced[y_train_balanced == 1])}, Negativos: {len(y_train_balanced[y_train_balanced == 0])}")


Train - Positivos después de aumentación: 304, Negativos: 400


In [31]:
# Modelo con MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False

modelo = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])


In [32]:
# Compila el modelo
modelo.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])


In [33]:
# Entrena el modelo
modelo.fit(X_train_balanced, y_train_balanced, epochs=2, batch_size=32, validation_data=(X_val, y_val), verbose=1)


Epoch 1/2
22/22 ━━━━━━━━━━━━━━━━━━━━ 125s 5s/step - accuracy: 0.9272 - loss: 0.1885 - val_accuracy: 0.4451 - val_loss: 4.4144
Epoch 2/2
22/22 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 0.9926 - loss: 0.0516 - val_accuracy: 0.4499 - val_loss: 3.1368


In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [35]:
# Calcula métricas
predicciones = (modelo.predict(X_val) > 0.5).astype(int).flatten()
accuracy = modelo.evaluate(X_val, y_val, verbose=0)[1]
accuracy_porcentaje = accuracy * 100
precision = precision_score(y_val, predicciones)
recall = recall_score(y_val, predicciones)
f1 = f1_score(y_val, predicciones)

print("\nResultados en validación:")
print(f"Precisión (Accuracy): {accuracy_porcentaje:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

97/97 ━━━━━━━━━━━━━━━━━━━━ 50s 471ms/step

Resultados en validación:
Precisión (Accuracy): 44.99%
Precision: 0.7714
Recall: 0.0157
F1-Score: 0.0307


In [20]:
 #Separa imágenes positivas y negativas
positivas_idx = np.where(y_train == 1)[0]
negativas_idx = np.where(y_train == 0)[0]
X_positivas = X_train[positivas_idx]
X_negativas = X_train[negativas_idx]

print(f"Train - Positivos originales: {len(X_positivas)}, Negativos originales: {len(X_negativas)}")

Train - Positivos originales: 5, Negativos originales: 1195


In [21]:
# Toma solo 200 negativos
np.random.seed(42)
negativas_seleccionadas_idx = np.random.choice(negativas_idx, size=200, replace=False)
X_negativas_seleccionadas = X_train[negativas_seleccionadas_idx]
# Aumentación para llegar a 300 positivos
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

In [22]:
X_positivas_aumentadas = []
y_positivas_aumentadas = []
target_positivas = 300
for i in range(len(X_positivas)):
    img = X_positivas[i]
    img = img.reshape((1,) + img.shape)
    count = 0
    for batch in datagen.flow(img, batch_size=1):
        X_positivas_aumentadas.append(batch[0])
        y_positivas_aumentadas.append(1)
        count += 1
        if len(X_positivas_aumentadas) >= target_positivas - len(X_positivas):
            break
    while len(X_positivas_aumentadas) < target_positivas - len(X_positivas) and count < 100:
        for batch in datagen.flow(img, batch_size=1):
            X_positivas_aumentadas.append(batch[0])
            y_positivas_aumentadas.append(1)
            count += 1
            if len(X_positivas_aumentadas) >= target_positivas - len(X_positivas):
                break
# Combina datos
X_train_balanced = np.concatenate([X_negativas_seleccionadas, X_positivas, np.array(X_positivas_aumentadas)])
y_train_balanced = np.concatenate([np.zeros(200), np.ones(len(X_positivas)), np.ones(len(X_positivas_aumentadas))])

print(f"Train - Positivos después de aumentación: {len(y_train_balanced[y_train_balanced == 1])}, Negativos: {len(y_train_balanced[y_train_balanced == 0])}")
print(f"Forma de X_train_balanced: {X_train_balanced.shape}")

Train - Positivos después de aumentación: 304, Negativos: 200
Forma de X_train_balanced: (504, 128, 128, 3)


In [23]:
# Modelo con MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

modelo = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [24]:
# Compila el modelo
modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [25]:
# Entrena el modelo
history = modelo.fit(X_train_balanced, y_train_balanced, epochs=30, batch_size=32, validation_data=(X_val, y_val), verbose=1)

Epoch 1/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 121s 6s/step - accuracy: 0.8293 - loss: 0.3788 - val_accuracy: 0.6137 - val_loss: 0.7568
Epoch 2/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.9914 - loss: 0.0360 - val_accuracy: 0.6214 - val_loss: 0.8030
Epoch 3/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.9983 - loss: 0.0112 - val_accuracy: 0.6269 - val_loss: 0.7815
Epoch 4/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 49s 3s/step - accuracy: 0.9953 - loss: 0.0122 - val_accuracy: 0.6289 - val_loss: 0.7647
Epoch 5/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.9998 - loss: 0.0075 - val_accuracy: 0.6308 - val_loss: 0.7631
Epoch 6/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.9987 - loss: 0.0073 - val_accuracy: 0.6305 - val_loss: 0.7676
Epoch 7/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 65s 4s/step - accuracy: 0.9986 - loss: 0.0067 - val_accuracy: 0.6328 - val_loss: 0.7688
Epoch 8/30
16/16 ━━━━━━━━━━━━━━━━━━━━ 59s 4s/step - accuracy: 0.9991 - loss: 0.0048 - val_accuracy: 0.6286 - val_loss

In [26]:
# Calcula métricas con umbral ajustable
for umbral in [0.5, 0.3, 0.2]:
    predicciones = (modelo.predict(X_val) > umbral).astype(int).flatten()
    accuracy = np.mean(predicciones == y_val) * 100
    precision = precision_score(y_val, predicciones)
    recall = recall_score(y_val, predicciones)
    f1 = f1_score(y_val, predicciones)
    
    print(f"\nResultados con umbral {umbral}:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")


97/97 ━━━━━━━━━━━━━━━━━━━━ 44s 413ms/step

Resultados con umbral 0.5:
Accuracy: 56.52%
Precision: 0.7717
Recall: 0.3101
F1-Score: 0.4424
97/97 ━━━━━━━━━━━━━━━━━━━━ 40s 404ms/step

Resultados con umbral 0.3:
Accuracy: 59.33%
Precision: 0.7365
Recall: 0.4187
F1-Score: 0.5339
97/97 ━━━━━━━━━━━━━━━━━━━━ 40s 401ms/step

Resultados con umbral 0.2:
Accuracy: 61.98%
Precision: 0.7239
Recall: 0.5116
F1-Score: 0.5995


In [27]:
# Modelo con MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

modelo = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [28]:
# Compila el modelo
modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               loss='binary_crossentropy',
               metrics=['accuracy'])

In [29]:
# Entrena el modelo
history = modelo.fit(X_train_balanced, y_train_balanced, epochs=35, batch_size=32, validation_data=(X_val, y_val), verbose=1)

Epoch 1/35
16/16 ━━━━━━━━━━━━━━━━━━━━ 103s 5s/step - accuracy: 0.7697 - loss: 0.4069 - val_accuracy: 0.5155 - val_loss: 0.8841
Epoch 2/35
16/16 ━━━━━━━━━━━━━━━━━━━━ 50s 3s/step - accuracy: 0.9836 - loss: 0.0511 - val_accuracy: 0.5320 - val_loss: 0.9218
Epoch 3/35
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.9962 - loss: 0.0191 - val_accuracy: 0.5304 - val_loss: 0.9065
Epoch 4/35
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.9873 - loss: 0.0228 - val_accuracy: 0.5313 - val_loss: 0.9071
Epoch 5/35
16/16 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.9926 - loss: 0.0170 - val_accuracy: 0.5417 - val_loss: 0.8904
Epoch 6/35
16/16 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - accuracy: 0.9996 - loss: 0.0080 - val_accuracy: 0.5478 - val_loss: 0.8942
Epoch 7/35
16/16 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step - accuracy: 0.9982 - loss: 0.0069 - val_accuracy: 0.5475 - val_loss: 0.8968
Epoch 8/35
16/16 ━━━━━━━━━━━━━━━━━━━━ 48s 3s/step - accuracy: 0.9984 - loss: 0.0098 - val_accuracy: 0.5484 - val_loss

In [30]:
# Calcula métricas con umbral ajustable
for umbral in [0.5, 0.3, 0.2, 0.1]:
    predicciones = (modelo.predict(X_val) > umbral).astype(int).flatten()
    accuracy = np.mean(predicciones == y_val) * 100
    precision = precision_score(y_val, predicciones)
    recall = recall_score(y_val, predicciones)
    f1 = f1_score(y_val, predicciones)
    
    print(f"\nResultados con umbral {umbral}:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")


97/97 ━━━━━━━━━━━━━━━━━━━━ 46s 429ms/step

Resultados con umbral 0.5:
Accuracy: 46.03%
Precision: 0.7107
Recall: 0.0499
F1-Score: 0.0933
97/97 ━━━━━━━━━━━━━━━━━━━━ 40s 400ms/step

Resultados con umbral 0.3:
Accuracy: 49.16%
Precision: 0.7587
Recall: 0.1260
F1-Score: 0.2161
97/97 ━━━━━━━━━━━━━━━━━━━━ 42s 404ms/step

Resultados con umbral 0.2:
Accuracy: 50.61%
Precision: 0.7506
Recall: 0.1678
F1-Score: 0.2743
97/97 ━━━━━━━━━━━━━━━━━━━━ 41s 405ms/step

Resultados con umbral 0.1:
Accuracy: 53.17%
Precision: 0.7329
Recall: 0.2485
F1-Score: 0.3712
